# Steps
1. Set Translation Language
2. upload one json to translate
3. run all cells
4. download your translation


# TODO:
make simple version?

edge cases:
- unicode characters and UTF8 conversion
- empty values
- broken string handling: single character values,



##Fancier version will:
4. connect open-ai
5. connect gguf
6. remove any \u00e8 type characters
7. check for empty leaf node (as fail json test)

maybe:
- not do whole sections but iterate
item by items crawl...



Version:
1. python script
2. endpoint
3. using api-model service (online)
4. using local .GGUF model (offline)
	- make:
		A. raw llama.cpp wrapper
		B. call_api for python-lamma-cpp

...

for writeup:
discuss balance of system-1 and system -2
remove any \u00e8 type characters




In [ ]:
"""
.env: get your environment variables:
  Using the Google Secretes (like.env) system
  built into colab on the left menu: the 'key' icon.
"""
from google.colab import userdata
mistral_api_key = userdata.get('mistral_api_key')
# mistral_api_key = 'xxx'
openai_api_key = userdata.get('open_ai_key')

# make a list of json files

In [ ]:
import os


# Helper Function
def list_json_files_in_cwd():
    """
    Returns a list of .json files in the current working directory.
    """
    # List all files in the current working directory
    files_in_cwd = os.listdir('.')

    # Filter the list to include only .json files
    json_files = [file for file in files_in_cwd if file.endswith('.json')]

    return json_files


# Process Json
- make a model/skeleton template

In [ ]:
import json


# Helper Function
def load_json_file(file_path):
    """
    Loads and reads a JSON file, returning its content as a Python dictionary.

    :param file_path: Path to the .json file to be read.
    :return: A dictionary containing the data from the JSON file.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


# Helper Function
def create_deep_empty_structure(data):
    """
    Recursively creates a deep copy of the provided data structure,
    replacing all terminal values with empty lists, and maintaining the same structure.

    :param data: The original data structure, potentially containing nested dictionaries.
    :return: A new data structure with the same keys but with empty lists as terminal values.
    """
    if isinstance(data, dict):  # Checks if the current data item is a dictionary
        return {key: create_deep_empty_structure(value) for key, value in data.items()}
    else:
        return []  # Replaces non-dictionary values with an empty list


# Helper Function
def create_empty_json_file(original_data, new_file_path):
    """
    Creates a new JSON file replicating the structure of the original data,
    but with empty lists for all terminal values.

    :param original_data: Dictionary containing the original data.
    :param new_file_path: Path for the new .json file to be created.
    """
    # Create a new structure with the same keys but with empty lists as terminal values
    empty_data = create_deep_empty_structure(original_data)

    # # Write this new structure to a new JSON file
    # with open(new_file_path, 'w') as file:
    #     json.dump(empty_data, file, indent=4)

    return empty_data


# put translation into list_skeleton_json

In [ ]:
# Helper Function
def populate_skeleton_json_with_data(skeleton_json, source_data):
    """
    Recursively fills the skeleton_json structure with data from the source_data.
    For lists, it merges items instead of replacing them, ensuring unique values.

    :param skeleton_json: The skeleton_json structure with potentially empty lists as terminal values.
    :param source_data: The source data structure from which values are to be copied.
    :return: The skeleton_json structure filled with values from source_data, with care for unique list items.
    """
    if isinstance(skeleton_json, dict) and isinstance(source_data, dict):
        for key in skeleton_json:
            if key in source_data:
                # Recurse into nested dictionaries or replace/merge list and terminal values
                skeleton_json[key] = populate_skeleton_json_with_data(skeleton_json[key], source_data[key])
    elif isinstance(skeleton_json, list):
        # Ensure unique items are added from source_data to skeleton_json list, preserving existing items.
        skeleton_json.extend([item for item in source_data if item not in skeleton_json])
        return skeleton_json
    elif isinstance(skeleton_json, list) and not skeleton_json:
        # For an empty list in skeleton_json, directly copy the source_data list to it
        return source_data.copy()
    # For terminal values, or if the structures don't match (dict/list), return source_data to replace or fill in.
    else:
        return source_data
    return skeleton_json


## save json utility

In [ ]:
import os
from datetime import datetime as dt
import json  # Added missing import

# Helper Function
def save_json_to_file(input_text, file_name, target_language, optional_tag=""):
    """
    Saves a JSON object to a file.

    :param data: The JSON object to save.
    :param file_path: The path to the JSON file where the object should be saved.
    """
    # with open(file_path, 'w') as file:
    #     json.dump(data, file, indent=4)

    # make readable time
    date_time = dt.utcnow()
    clean_timestamp = date_time.strftime('%Y%m%d%H%M%S%f')

    new_title = f"{target_language}_{clean_timestamp}_{file_name}"

    new_title = optional_tag + new_title

    # Determine the path to the directory that should contain the file
    directory_path = 'translations'

    # Check if the directory exists
    if not os.path.exists(directory_path):

        # If it does not exist, create it
        # Ensure the directory exists
        try:
            os.makedirs(directory_path, exist_ok=True)  # Ensure the directory is created if it does not exist
        except Exception as e:
            print(f"Error creating directory {directory_path}: {e}")
            return  # Exit the function if directory creation fails

    # Determine the path to the file that should be saved
    file_path = os.path.join(directory_path, new_title)

    # Save the JSON data to the file with UTF-8 encoding
    with open(file_path, 'w', encoding='utf-8') as outfile:
        json.dump(input_text, outfile, indent=4, ensure_ascii=False)

# Functions to call Mistral

In [ ]:
"""
related tools:

api throttle_protection
api async

import requests
"""
import requests
import json
import os
import re
import time

# using google secretes
from google.colab import userdata
mistral_api_key = userdata.get('mistral_api_key')

# Define the endpoint URL
endpoint_url = "https://api.mistral.ai/v1/chat/completions"

# Set the headers
headers = {
  "Content-Type": "application/json",
  "Accept": "application/json",
  "Authorization": f"Bearer {mistral_api_key}"
}

"""
# mode: [{"role": "user", "content": "say yes"}]

    # Define the request body
    request_body = {
      "model": "mistral-small",
      "messages": [{"role": "user", "content": user_input}]
    }

    # Send the request
    response = requests.post(endpoint_url, headers=headers, json=request_body)
"""


# Helper Function
def segment_for_adding_to_context_history(role, comment):

    if role == 'user':
        segment = {"role": "user", "content": comment}

    elif role == 'assistant':
        segment = {"role": "assistant", "content": comment}

    elif role == 'system':
        segment = {"role": "system", "content": comment}

    else:
        print("error segment_for_adding_to_context_history(role, comment)")
        print(role, comment)
        print('error')

    return segment


# Helper Function
def add_to_context_history(user_input, context_history, role):

    if role == 'user':
        segment = {"role": "user", "content": user_input}

    elif role == 'assistant':
        segment = {"role": "assistant", "content": user_input}

    elif role == 'system':
        segment = {"role": "system", "content": user_input}

    else:
        print('error add_to_context_history() error')
        print(role, user_input)
        print('role error?')

    context_history.append( segment )

    return context_history


# Helper Function
def prompt_user(user_input, context_history):

    context_history.append( segment_for_adding_to_context_history("user", user_input) )

    return context_history


# Helper Function
def one_response_to_user(user_input, context_history, use_this_model):
    """
    Input: context_history
    Ouput Tuple: assistant_says, context_history
    """

    counter(3)

    # prompt user
    context_history = prompt_user(user_input, context_history)

    # prompt assistant
    response = ask_mistral_tiny(context_history, use_this_model)

    return response


# Helper Function
def counter(timeout=10):
    count = 0
    start_time = time.time()
    while time.time() - start_time < timeout:
        count += 1
        time.sleep(1)
    return count


# Helper Function
def ask_mistral_tiny(context_history, use_this_model):
    # Define the request body
    request_body = {
      "model": use_this_model,
      "messages": context_history
    }

    #################
    #################
    # Hit the ai api
    #################
    #################
    # Send the request
    response = requests.post(endpoint_url, headers=headers, json=request_body)

    # Check the response status code
    if response.status_code != 200:
        print(f"Error: {response.status_code} {response.text}")
        return None

        # pause if error occurs
        counter(10)

    # Get the response data
    response_data = response.json()

    # Print the Mistral response

    ##
    ##
    # Turn this print on to see full return data
    ##
    ##
    """
    e.g.
    {
      "id": "635cb8d445ujhe5546bb64e5e7",
      "object": "chat.completion",
      "created": 170hrjfjf7084,
      "model": "mistral-tiny",
      "choices": [
        {
          "index": 0,
          "message": {
            "role": "assistant",
            "content": "Enjoy your cup of tea!"
          },
          "finish_reason": "stop",
          "logprobs": null
        }
      ],
      "usage": {
        "prompt_tokens": 575,
        "total_tokens": 629,
        "completion_tokens": 54
      }
    }
    """
    # print(json.dumps(response_data, indent=2))
    # print(type(response_data))

    output = response_data
    # print(type(output))
    # print(type(output["choices"][0]))

    # extract just the 'what they said' part out
    assistant_says = output["choices"][0]['message']['content']


    return assistant_says


# functions to call openAI api

In [ ]:
!pip install -q openai

In [ ]:
# functions to call openAI 2024 api


import openai
import time
from openai import AsyncOpenAI

# my_api_key = ""

from google.colab import userdata
my_api_key = userdata.get('open_ai_key')

from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key = my_api_key,
)

"""
api module parts
"""

import string
import asyncio
from openai import AsyncOpenAI
import threading

############
# Not Async
############
"""
openai_call_context_timeout() with x-retries and time-limit freeze protection
"""


def container_api_call(client, model, temp, context, result_container):
    try:
        completion = client.chat.completions.create(
            model=model,
            temperature=temp,
            messages=context
        )
        result_container['completion'] = completion
    except Exception as e:
        result_container['error'] = e


def run_api_call_with_timeout(client, model, temp, context, timeout):
    """
    flags the thread and moves to background after time-out
    but does not attempt to 'kill' the thread.
    If thread finishes, output ~should stay in background
    un-unused by anything.
    """
    try:
        result_container = {}
        api_thread = threading.Thread(target=container_api_call, args=(client, model, temp, context, result_container))
        api_thread.start()
        api_thread.join(timeout)

        if api_thread.is_alive():
            # If the API call is still running after the timeout
            print("The API call timed out.")
            return None  # Or handle the timeout situation as needed

        if 'error' in result_container:
            # If there was an error during the API call
            print("An error occurred:", result_container['error'])
            return None

        return result_container.get('completion')

    except Exception as e:
        raise( e, f"ERROR in function: run_api_call_with_timeout(input_string)" )


# helper function ()
def openai_call_context_timeout(client, context, model="gpt-4", max_retries=10, temp=0.9, timeout_min=8):
    """
    requires: container_api_call(), run_api_call_with_timeout()
    """
    # load_dotenv()
    # openai.api_key = os.getenv("OPENAI_API_KEY")

    # convert timeout into seconds (int)
    timeout_sec = int(timeout_min * 60)

    # makes sure number is not zero
    if timeout_sec <= 0:
        timeout_sec = 1

    """
    Time parameters for retry to double in duration until the max seconds limit
    """
    base_delay_sec = 2
    max_delay_sec = 16

    retries = 0
    while retries <= max_retries:
        try:
            # call API within a time-out filter
            completion = run_api_call_with_timeout(
                client,
                model=model,
                temp=temp,
                context=context,
                timeout=timeout_sec
            )

            text = completion.choices[0].message.content

            print(f"openai_call_context_timeout() -> {text}")

            return text

        except Exception as e:
            print(f"an error occurred: {e}")
            retries += 1
            if retries > max_retries:
                print("max retries reached. exiting.")
                raise

            # using min() to ensure that delay <= max
            real_delay_sec = min(max_delay_sec, (2 ** retries) * base_delay_sec)
            print(f"retrying in {real_delay_sec} seconds...")
            # pause before retrying
            time.sleep(real_delay_sec)
            """
            Note: time delay is not a crash-exception, it is a re-try
            """




In [ ]:
# # test
# context = []

# # "system"
# system_prompt = f"""You are a talking bird."""

# context.append({
#     "role": "system", "content": system_prompt
# })

# prompt = "Squawk, I am a bird."
# context.append({
#     "role": "assistant", "content": prompt
# })

# prompt = "what is a bird?"
# context.append({
#     "role": "user", "content": prompt
# })

# print(context)

# # result = openai_call_context(context, model="gpt-4", max_retries=2)

# # result = await openai_call_context_timeout_async(client, context, model="gpt-3.5-turbo", max_retries=2, temp=0.9, timeout_min=2)

# # print(f"result -> {result}")


In [ ]:

# client = OpenAI(
#     # defaults to os.environ.get("OPENAI_API_KEY")
#     api_key = my_api_key,
# )


# pause_time_seconds = .5

# # Helper Function
# def pause_ok():
#     time.sleep(pause_time_seconds)
#     return "Pause OK!"

# # Helper Function: Call API Function
# def call_openai_chat_api(this_input, select_model=3):
#     """
#     Requires:
#       import time
#       pip install openai
#       import openai
#       api key from openai, included in subscription:
#       https://platform.openai.com/account/api-keys

#       # Helper Function
#       def pause_ok():
#           time.sleep(pause_time_seconds)
#           return "OK!"


#     Chat type query to openAI:
#     pecify Model, default is "gpt-3.5-turbo"
#     user input-2 selects which model to use: 3, 4

#     Sample use:

#     this_input = "Hello."
#     select_model = 3

#     call_openai_chat_api(this_input, select_model=3)
#     """

#     try:
#         if select_model == 4:
#             this_model = "gpt-4"

#         elif select_model == "gpt-4":
#             this_model = "gpt-4"

#         elif select_model == "gpt-3.5-turbo":
#             this_model = "gpt-3.5-turbo"

#         elif select_model == 3:
#             this_model = "gpt-3.5-turbo"

#         # Pause to avoid throttling
#         print(f"Pausing {pause_time_seconds} seconds...")
#         print(pause_ok())
#         # time.sleep(pause_time_seconds)
#         # print(f"Pausing {pause_time_seconds} seconds...OK!")


#         ## Hit API
#         completion = client.chat.completions.create(model=this_model, messages=[{"role": "user", "content": this_input}])

#         # Terminal Inspection for Examination (optional)
#         # print(completion)

#         ## Results (slicing into the json object output)
#         results = completion.choices[0].message.content

#     except Exception as e:
#         print("API call function call_openai_chat_api(this_input, select_model=3) failed, error = ", e)
#         return e

#     return results


In [ ]:
# call_openai_chat_api("testing")

## Add System instructions for rules

In [ ]:
# Helper Function
def set_translator__system_prompt(context_history, target_language, skeleton_json):

    ################
    # System Prompt
    ################

    example_1 = {
    "Form_One": {
    "FirstName":"*First Name*",
    "LastName":"*Last Name*",
    "Email":"*Email*",}
    }
    example_2 = {
    "Form_One": {
    "FirstName":"TRANSLATED_First_Name",
    "LastName":"TRANSLATED_Last Name",
    "Email":"TRANSLATED_Email",}
    }

    example_json = {
      "yes": {
      "key": "value",
      "key2": "value2"
      }
      }

    example_json2 = {
      'BAD': {
      'NO NO NO': 'NO SINGLE QUOTES',
      'NOT THIS': 'NO SINGLE QUOTES'
      }
      }

    # set translation language and structure of output in system
    text_input = f"""
    You are a helpful {target_language} language translator bot that produces high
    quality professional translations in precise json formats.

    You translate writen language, not emojis or syntax not
    readable by a person. You use normal phrase or sentence capitalization,
    not all upper case. You use normal full words, not single letters
    or obscure abreviations.

    You only translate the leaf values or terminal values in the json.
    e.g. If the original json says:
    {example_1}
    Then you only translate the final leaf values, NOT the keys above:
    {example_2}

    Your translation is always expressed using valid json syntax,
    using double quotes NOT single quotes:
    as:
    ```json
    {example_json}
    ```
    NOT THIS:
    {example_json2}

    You only translate into {target_language}.
    You only produce json output in exactly this structure {skeleton_json}.
    Your translations are clear, accurate, helpful, honrable, brief, polite, and professional.
    Your do you best to tranlsate every leaf value field leaving nothing blank.
    Every final leaf values MUST be translated.

    You always double check your work and make sure the translation is
    excellent in the context of the whole body of translation.
    """
    role = "system"

    context_history.append( segment_for_adding_to_context_history(role, text_input) )

    # # inspection
    # print("set_translator__system_prompt -> ", context_history)

    return context_history


## Add 'user' request for translation

In [ ]:
# Helper Function
def set_translate__user_prompt(context_history, target_language, skeleton_json, original_data):

    ###########################
    # User Translation Request
    ###########################

    example_json = {
      "body": {
      "key": "value",
      "key2": "value2"
      }
      }

    example_json2 = {
      'BAD': {
      'NO NO NO': 'NO SINGLE QUOTES',
      'NOT THIS': 'NO SINGLE QUOTES'
      }
      }

    # set translation language and structure of output in system
    text_input = f"""

    The person we are trying to help needs a json file
    translated into the {target_language} language.
    Let's help them translate the values in their json file.

    Carefully translate this original file into {target_language}
    {original_data}.

    Double check your work and make sure the translation is
    accurate, brief, and polite.

    Make sure your translation is expressed using valid json syntax,
    using double quotes NOT single quotes:
    as:
    ```json
    {example_json}
    ```
    NOT THIS:
    {example_json2}

    (e.g. no trailing delimiter, escape conflicting characters, etc).
    The structure of your translation must fit this structure
    exactly: {skeleton_json}

    You can do it!!
    """
    role = "user"

    context_history.append( segment_for_adding_to_context_history(role, text_input) )

    # # inspection
    # print("set_translate__user_prompt", context_history)

    return context_history



# json inspection

In [ ]:
# Helper Function
def dict_leaf_detection_boolean_true_means_defective(input_dict):
    """
    Recursively checks if any terminal leaf node in a nested structure is an empty string,
    an empty list, or a single character.

    :param input_dict: The nested input_dict structure to check.
    :return: True if any leaf node is an empty string, an empty list, or a single character; False otherwise.
    """
    if isinstance(input_dict, dict):  # If the current item is a dictionary
        return any(dict_leaf_detection_boolean_true_means_defective(value) for value in input_dict.values())
    elif isinstance(input_dict, list):  # If the current item is a list
        return any(dict_leaf_detection_boolean_true_means_defective(item) for item in input_dict)
    else:  # If the current item is a leaf node
        return input_dict == "" or (isinstance(input_dict, list) and len(input_dict) == 0) or (isinstance(input_dict, str) and len(input_dict) == 1)


In [ ]:
# Helper Function
def check_function_description_keys(dict_str, model_dict):
    """
    This function CAN fail and should fail
    if the AI needs to retry at a task.
    Do not stop server when this this triggers an exception.

    edge case: before there is a populated output_log

    if passing, this function will return a valid json object
    """

    """
    Extracts JSON string enclosed between ```json and ``` markers.

    Parameters:
    - text (str): The input text containing the JSON block.

    Returns:
    - str: The extracted JSON string, or an empty string if no JSON block is found.
    """
    print(f"\n\n Starting check_function_description_keys, dict_str -> {dict_str}")

    try:
        # if test fails
        if dict_leaf_detection_boolean_true_means_defective(dict_str):
            return False

    except Exception as e:
        print(f"\nTRY AGAIN: dict_leaf_detection_boolean_true_means_defective() empty or stub leaf found: {e}")
        print(f"Failed dict_str -> {dict_str}")
        return False


    ########################
    # Check Json Formatting
    ########################
    try:
        pattern = r'```json\n([\s\S]*?)\n```'
        match = re.search(pattern, dict_str)
        dict_str =  match.group(1) if match else ''

    except Exception as e:
        print(f"\nTRY AGAIN: check_function_description_keys() extraction from markdown failed: {e}")
        print(f"Failed dict_str -> {dict_str}")
        return False

    print(f"\n extracted from markdown ->{dict_str}")

    # clean
    try:
        # try safety cleaning
        dict_str = dict_str.replace("True", "true")
        dict_str = dict_str.replace("False", "false")
        dict_str = dict_str.replace("None", "null")

        # # This conflicted with free language in description section...
        # dict_str = dict_str.replace("'", '"')

        # remove trailing delimiter comma
        print(f"{dict_str[:-6]}")
        dict_str = dict_str.replace('",\n}', '"\n}')

    except Exception as e:
        print(f"\nTRY AGAIN:try safety cleaning: {e}")
        print(f"Failed dict_str -> {dict_str}")
        return False

    # load
    try:
        # try converting
        dict_str = json.loads(dict_str)

    except Exception as e:
        print(f"\nTRY AGAIN: trying json.loads(dict_str) Dictionary load failed: {e}")
        print(f"Failed dict_str -> {dict_str}")
        return False


    # check if keys are the same
    try:
        result = dict_str.keys() == model_dict.keys()
        if result is False:
            print(f"Failed: keys are not the same.")
            print(f"Failed dict_str -> {dict_str}")
            return False

    except Exception as e:
        print(f"\nTRY AGAIN: Failed with Exception: keys are not the same: {e}")
        print(f"Failed dict_str -> {dict_str}")
        return False

    # if ok...
    return dict_str



# Call api within: Check json structure against original

In [ ]:
# helper function
def call_api_within_structure_check(context_history, use_this_model, mode_locale, skeleton_json):
    retry_counter = 0
    json_ok_flag = False

    # see
    mistal_model_list = [
        "mistral-tiny",
        "mistral-small",
        "mistral-large-latest",
    ]

    # see https://platform.openai.com/docs/guides/text-generation
    open_ai_model_list = [
        "gpt-4",
        "gpt-4-turbo-preview",
        "gpt-3.5-turbo"
    ]


    gguf_model_list = [
        "jais",
        "tiny_llama",
        "mistral8x7...",
        ""
    ]

    while not json_ok_flag:

        ####################
        # get a translation
        ####################

        try:
            # check json structure

            ########################
            # Select Model and Mode
            ########################

            # for off-line local mode
            if mode_locale == "gguf":
                """
                TODO: code for using local gguf model
                """
                pass

            ################
            # for cloud api
            ################
            elif use_this_model in mistal_model_list:
                print(f"Mistral api selected...{use_this_model}")
                dict_str = ask_mistral_tiny(context_history, use_this_model)

            elif use_this_model in open_ai_model_list:
                print(f"openAI api selected...{use_this_model}")
                dict_str = openai_call_context_timeout(client, context_history, model=use_this_model, max_retries=10, temp=0.9, timeout_min=8)

            else:
                print(f"no known api selected...{use_this_model}")
                raise f"No known model option chosen...use_this_model -> {use_this_model}"


        except Exception as e:
            jsonchecked_translation = None
            print(f"Failed: {str(e)}")

        jsonchecked_translation = check_function_description_keys(dict_str, skeleton_json)

        if jsonchecked_translation:
            json_ok_flag = True

        else:
            retry_counter += 1

    print(f"retry_counter -> {retry_counter}")

    return jsonchecked_translation


# Put Flesh on the Skeleton
- add translations to the lists

In [ ]:
# helper function
def set_select_best__system_prompt(context_history, target_language, skeleton_json, original_data):

    ################
    # System Prompt to pick best traslation
    ################

    example_1 = {
    "Form_One": {
    "FirstName":[],
    "LastName":[],
    "Email":[],}
    }
    example_2 = {
    "Form_One": {
    "FirstName":"TRANSLATED_First_Name",
    "LastName":"TRANSLATED_Last Name",
    "Email":"TRANSLATED_Email",}
    }



    # set translation language and structure of output in system
    text_input = f"""
    You are a helpful translator bot that selects high
    quality professional translations from a list and produces or reproduces precise json formats.


    You select the one best translate in the {target_language}.
    You only produce json output in exactly this structure {skeleton_json},
    but with a leaf node text not a list.


    e.g. The list option json will have this form:
    {example_1}
    And your final form must contain one best translation, not a list.
    {example_2}


    Your final translation selections are clear, accurate, helpful, honrable, brief, polite, and professional.
    Your do you best to tranlsate every leaf value field leaving nothing blank.

    You always double check your work and make sure the translation is
    excellent in the context of the whole body of translation.
    """
    role = "system"

    context_history.append( segment_for_adding_to_context_history(role, text_input) )

    return context_history



In [ ]:
# helper function
def set_select_best__user_prompt(context_history, target_language, skeleton_json, populated_skeleton, original_data):

    ###########################
    # User Translation Request to select best translation
    ###########################

    # set translation language and structure of output in system
    text_input = f"""

    The person we are trying to help
    selecting the best {target_language} language translation for their final translated json file.
    Let's help them select the best {target_language} language translation leaf node values for their json file.

    Task to do: Carefully the best {target_language} language translation leaf node values
    from this list_of_translations json {populated_skeleton}.

    Double check your work and make sure the translation is
    accurate, brief, and polite.

    Make sure your translation is expressed using valid json syntax,
    as:
    ```json
    JSON_OBJECT(S)
    ```

    (e.g. no trailing delimiter, escape conflicting characters, etc).
    The structure of your translation must fit this structure
    {skeleton_json} but with a single best translation not a list.
    """
    role = "user"

    context_history.append( segment_for_adding_to_context_history(role, text_input) )

    # print("\n\n\n\nset_select_best__user_prompt() context_history", context_history)

    return context_history


In [ ]:
# Main Function

def translate_json(list_of_targeted_languages, use_this_model, mode_locale, number_of_preliminary_translations=3):

    ######################
    # Translation Factory
    ######################

    # Example usage
    json_files_list = list_json_files_in_cwd()

    if not json_files_list:
        print("Error: You missed a step, No Json files were provided.")
        raise "No Json files were provided."

    # inspection
    print("JSON files in the CWD:", json_files_list)

    for this_json_file in json_files_list:

        # Load the original JSON file
        original_data = load_json_file(this_json_file)

        # Create a new JSON file with the deep empty structure
        name_of_skeleton_saved_file = 'empty_lists_' + '_' +this_json_file
        skeleton_json = create_empty_json_file(original_data, name_of_skeleton_saved_file)

        for target_language in list_of_targeted_languages:

            ################################
            # Make preliminary Translations
            ################################
            populated_skeleton = skeleton_json

            for i in range(number_of_preliminary_translations):

                # make empty conversation
                # reset context history for new 'conversation' about translation
                context_history = []

                # Set Prompts per one language

                # System Instructions
                context_history = set_translator__system_prompt(context_history, target_language, skeleton_json)
                # User Prompt
                context_history = set_translate__user_prompt(context_history, target_language, skeleton_json, original_data)

                # # # inspection breakpoint
                # print(f"\n\n breakpoint 2: context_history -> {context_history}")
                # input("breakpoint")

                ############
                # Translate
                ############
                jsonchecked_translation = call_api_within_structure_check(context_history, use_this_model, mode_locale, skeleton_json)

                # add translations to populated skeleton
                populated_skeleton = populate_skeleton_json_with_data(populated_skeleton, jsonchecked_translation)

                # # inspection breakpoint
                # print(f"\n\n breakpoint 4: populated_skeleton -> {populated_skeleton}")
                # input("breakpoint")


            #####################################################
            # Select Top Top Goodest Translation Star-Good-Prime
            #####################################################

            # reset context history for new 'conversation' about selection
            context_history = []

            print("\n\n\nSelect Top Top Goodest Translation Star-Good-Prime")
            # # inspection breakpoint
            # print(f"\n\n breakpoint 5: populated_skeleton -> {populated_skeleton}")
            # # input("breakpoint")

            save_json_to_file(populated_skeleton, this_json_file, target_language, "set_of_translations_")

            # set prompts to select best translation

            # System Instructions
            context_history = set_select_best__system_prompt(context_history, target_language, skeleton_json, original_data)
            # User Prompt
            context_history = set_select_best__user_prompt(context_history, target_language, skeleton_json, str(populated_skeleton), original_data)


            ##############
            # Select Best
            ##############
            # # inspection
            # print(f"context_history -> {context_history}")
            # input("breakpoint")

            selected_translation_json = call_api_within_structure_check(context_history, use_this_model, mode_locale, skeleton_json)

            save_json_to_file(selected_translation_json, this_json_file, target_language, "selected_")


# Tranlate Json Files
- Set your language list
- Pick your model
- upload your json files
- Run
- Download / Get your translations (and inspection files if needed)

In [ ]:
##########################################################
# https://platform.openai.com/docs/guides/text-generation
##########################################################
"""
gpt-4

gpt-4-turbo-preview

gpt-3.5-turbo
"""

use_this_model = "gpt-4"
use_this_model = "gpt-4-turbo-preview"
use_this_model = "gpt-3.5-turbo"

In [ ]:
######################
# Mixtral 8x7 "Small"
######################
use_this_model = "mistral-small"

######################
# Mixtral Large ???
######################
use_this_model = "mistral-large-latest"

####################
# Mistral 7b "Tiny"
####################
use_this_model = "mistral-tiny"

In [ ]:
################################################
# Local Off-line Mode, or Online Cloud-api Mode
################################################

"""
"cloud_api"

"gguf"
"""

mode_locale = "cloud_api"

# Choices:

In [ ]:
# use_this_model = "gpt-3.5-turbo"
# use_this_model = "mistral-tiny"
use_this_model = "mistral-large-latest"


In [ ]:
mode_locale = "cloud_api"

In [ ]:
list_of_targeted_languages = ["Portuguese", "French", "German", "Italian", "Korean"]

In [ ]:
number_of_preliminary_translations = 3

In [ ]:
translate_json(list_of_targeted_languages, use_this_model, number_of_preliminary_translations)

JSON files in the CWD: ['beta_example_language_json_global.json']
Mistral api selected...mistral-large-latest


 Starting check_function_description_keys, dict_str -> Here is the json file translated into Portuguese:

```json
{
  "signUp": {
    "ConnectYour": ["CONECTE SEU"],
    "MindbankAiAccount": ["CONTA DA MINDBANK AI"],
    "FirstName": ["Primeiro Nome"],
    "LastName": ["Último Nome"],
    "Email": ["E-mail"],
    "PasswordMustHaveAtLeast8Characters": ["A senha deve ter pelo menos 8 caracteres"],
    "PasswordMustContainAtLeastOneNumber": ["A senha deve conter pelo menos um número"],
    "Password": ["Senha"],
    "ConfirmNewPassword": ["Confirme a Nova Senha"],
    "PasswordsDoNotMatch": ["As senhas não correspondem"],
    "YouMustAgreeToTheTerms&Conditions": ["Você deve concordar com os Termos & Condições"],
    "ByTickingTheBoxYourAreAgreeingToThe": ["Ao marcar o quadro, você está concordando com os"],
    "Terms&Conditions": ["Termos & Condições"],
    "SignUp": ["Cadastre

KeyboardInterrupt: 